## Koneksi

In [6]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = '127.0.0.1'
port = "3307"
user = 'root'
password = 'rahasia'
database = 'classicmodels'

engine = create_engine('mysql+pymysql://'+user+':'+password+'@'+host+':'+port+'/'+database)

# conn = pymysql.connect(
#     host=host,
#     port=int(port),
#     user=user,
#     passwd=password,
#     db=database,
#     charset='utf8mb4')

def run(sql):
    df = pd.read_sql_query(sql,engine)
    return df

## Baca File CSV

In [8]:
file_p = 'Data.csv'
data_df = pd.read_csv(file_p)
data_df.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


## Insert data to MySQL

In [32]:
data_df.to_sql('dt_0111',con=engine,if_exists='replace',index=False)
# fail / replace / append
#index false supaya index pada dataframe tidak masuk ke dalam mysql

## Fetch Data

In [31]:
engine.execute("select * from dt_0111").fetchall()

[('France', 44.0, 72000.0, 'No'),
 ('Spain', 27.0, 48000.0, 'Yes'),
 ('Germany', 30.0, 54000.0, 'No'),
 ('Spain', 38.0, 61000.0, 'No'),
 ('Germany', 40.0, None, 'Yes'),
 ('France', 35.0, 58000.0, 'Yes'),
 ('Spain', None, 52000.0, 'No'),
 ('France', 48.0, 79000.0, 'Yes'),
 ('Germany', 50.0, 83000.0, 'No'),
 ('France', 37.0, 67000.0, 'Yes')]

## Baca Data Dalam Bentuk Tabel

In [29]:
df_results = run("select * from dt_0111")
print(df_results)

   Country   Age   Salary Purchased
0   France  44.0  72000.0        No
1    Spain  27.0  48000.0       Yes
2  Germany  30.0  54000.0        No
3    Spain  38.0  61000.0        No
4  Germany  40.0      NaN       Yes
5   France  35.0  58000.0       Yes
6    Spain   NaN  52000.0        No
7   France  48.0  79000.0       Yes
8  Germany  50.0  83000.0        No
9   France  37.0  67000.0       Yes


In [7]:
run("""
SELECT 
    lastName, firstName
FROM
    employees
WHERE
    officeCode IN (SELECT 
            officeCode
        FROM
            offices
        WHERE
            country = 'USA')
""")

,lastName,firstName
0,Murphy,Diane
1,Patterson,Mary
2,Firrelli,Jeff
3,Bow,Anthony
4,Jennings,Leslie
5,Thompson,Leslie
6,Firrelli,Julie
7,Patterson,Steve
8,Tseng,Foon Yue
9,Vanauf,George


In [ ]:
run("""
SELECT 
    customerNumber, checkNumber, amount
FROM
    payments
WHERE
    amount = (SELECT 
            MAX(amount)
        FROM
            payments);
""")

In [ ]:
run("""
SELECT 
    customerNumber, checkNumber, amount
FROM
    payments
WHERE
    amount > (SELECT 
            AVG(amount)
        FROM
            payments);

""")

In [ ]:
run("""
SELECT 
    customerName
FROM
    customers
WHERE
    customerNumber NOT IN (SELECT DISTINCT
            customerNumber
        FROM
            orders);


""")

In [ ]:
run("""
SELECT 
    MAX(items), MIN(items), FLOOR(AVG(items))
FROM
    (SELECT 
        orderNumber, COUNT(orderNumber) AS items
    FROM
        orderdetails
    GROUP BY orderNumber) AS lineitems;

""")

In [ ]:
run("""
SELECT 
    customerNumber, 
    customerName
FROM
    customers
WHERE
    EXISTS( SELECT 
            orderNumber, SUM(priceEach * quantityOrdered)
        FROM
            orderdetails
                INNER JOIN
            orders USING (orderNumber)
        WHERE
            customerNumber = customers.customerNumber
        GROUP BY orderNumber
        HAVING SUM(priceEach * quantityOrdered) > 60000);

""")